# Convert GRIB2 to KMZ

In [14]:
import os
import shutil
import subprocess
from datetime import datetime, timedelta

from osgeo import gdal
import boto3

bucket_obj = boto3.resource("s3").Bucket("routewx")

# MAKE SURE TO CHANGE TO YOUR WORKING DIRECTORY SO GDAL KNOWS WHERE TO FIND YOUR FILE
#os.chdir("/workspace/routeweather-mapbox-ios/")

gdal.SetConfigOption("AWS_NO_SIGN_REQUEST", "YES")

retrieve_dt = datetime.utcnow() - timedelta(hours=2)
retrieve_hour = retrieve_dt.strftime("%H")
forecast_hour = (24 - int(retrieve_hour) if int(retrieve_hour) > 0 else 0) + 72
# Change to your filename here
vsis3_fname = f"/vsis3/noaa-nbm-grib2-pds/blend.{retrieve_dt:%Y%m%d}/{retrieve_hour}/core/blend.t{retrieve_hour}z.core.f0{forecast_hour}.co.grib2"

out_fname = "latest.kmz"
metadata = vsis3_fname.split("/")[-1]
band_vsi = "/vsimem/band.tif"
warp_vsi = "/vsimem/warp.tif"
color_tmp = "/tmp/colors.tif"
prefix = "nbm_snow_accum"


In [15]:
translate_options = gdal.TranslateOptions(bandList=[131])
print("extract desired band")
gdal.Translate(band_vsi, vsis3_fname, options=translate_options)

grbs = gdal.Open(band_vsi)

# reproject to GPS Coords
print("Reprojecting to (EPSG:4326)")
warp_options = gdal.WarpOptions(dstSRS="EPSG:4326", format="GTiff")
warp = gdal.Warp(warp_vsi, grbs, options=warp_options)

# apply custom color palette
print("Applying custom color palette to raster...")
dem_options = gdal.DEMProcessingOptions(colorFilename="snod.txt", format="GTiff", addAlpha=True)
gdal.DEMProcessing(color_tmp, warp, "color-relief", options=dem_options)

subprocess.run(["gdal_translate", "-co", "FORMAT=PNG", "-of", "KMLSUPEROVERLAY", "-dsco", "NameField=72-Hour Snow Accum", "-dsco", f"DescriptionField={metadata}", color_tmp, out_fname])

bucket_obj.upload_file(Filename=mbtiles_fname, Key=f"{prefix}/{out_fname}", ExtraArgs={"ACL": "public-read"})

gdal.Unlink(band_vsi)
gdal.Unlink(warp_vsi)
os.remove(color_tmp)

extract desired band
Reprojecting to (EPSG:4326)
Applying custom color palette to raster...
Input file size is 2942, 1404
0...10...20...30...40...50...60...70...80...90...100 - done.


In [16]:
import subprocess
from osgeo import gdal

#band = subprocess.check_output(["gdalinfo", "|", "blend.t15z.core.f033.co.grib2", "grep 'Total snowfall Percentile(50)' -B 4", "sed -n 's/.*Band//p'", "cut -d 'B' -f1", "xargs"])

#subprocess.run(["gdal_contour", "blend.t15z.core.f033.co.grib2", "-b", "85", "-i", "0.0254", "snow.kml"])
#gdaldem color-relief -nearest_color_entry -alpha -co format=png -of KMLSUPEROVERLAY raster.tif color.txt raster-kml.kmz
#print("Applying custom color palette to raster...")
#dem_options = gdal.DEMProcessingOptions(colorFilename="snod.txt", format="KMLSUPEROVERLAY", addAlpha=True, band=85)
#olors = gdal.DEMProcessing("snow.kmz", "blend.t15z.core.f033.co.grib2", "color-relief", options=dem_options)
#subprocess.run(["gdaldem", "color-relief", "-nearest_color_entry", "-b", "85", "-alpha", "-co", "format=png", "-of", "KMLSUPEROVERLAY", "blend.t15z.core.f033.co.grib2", "snod.txt", "snow.kmz"])
